In [1]:
import csv
import numpy as np

In [2]:
race_name = 'Test'
n_teams = 100
n_riders = 7

In [3]:
def initialise (n_teams, default):
    set = []
    for d in range(n_teams):
        set.append(default)
    return set

In [4]:
def assignment (set, comb, n_teams, index, default):
    new_set = initialise(n_teams, default)
    inter = set[0:index]
    
    if inter != []:
        new_set[0:index] = inter
        
    new_set[index] = comb
    
    for r in range(index+1, n_teams):
        new_set[r] = set[r-1]
    
    return new_set

In [5]:
def sorting (into, info_w):
    out_c = sorted([into[0], into[1]])
    out_s = sorted([into[2], into[3]])
    out_tt = sorted([into[4], into[5]])
                    
    for row in range(len(info_w)):
        if into[6] == info_w[row]['Name']:
            j = row
            break
    
    if j+1 <= n_c:
        out_c.append(into[6])
        out_c = sorted(out_c)
    elif j+1 <= n_c + n_s:
        out_s.append(into[6])
        out_s = sorted(out_s)
    else:
        out_tt.append(into[6])
        out_tt = sorted(out_tt)
        
    return out_c, out_s, out_tt

In [6]:
def checker (comb_riders, set_riders, info_w, n_teams):
    [comb_c_sort, comb_s_sort, comb_tt_sort] = sorting(comb_riders, info_w)
    
    for team in range(n_teams):
        if set_riders[team] != []:
            [set_c_sort, set_s_sort, set_tt_sort] = sorting(set_riders[team], info_w)
            unique = True
            counter = 0
            if comb_c_sort == set_c_sort:
                counter += 1
            if comb_s_sort == set_s_sort:
                counter += 1
            if comb_tt_sort == set_tt_sort:
                counter += 1
            if counter == 3:
                unique = False
                break
        else:
            unique = True
            
    return unique

In [7]:
# For Climbers
climber_file = open(race_name + '_Climber' + '.txt', 'r')
info_c = []
l_c = 0

data = csv.DictReader(climber_file)
for row in data:
    info_c.append(row)
    info_c[l_c]['Cost'] = int(info_c[l_c]['Cost'])
    info_c[l_c]['Points'] = int(info_c[l_c]['Points'])
    l_c += 1
       
climber_file.close()

# For Sprinters
sprinter_file = open(race_name + '_Sprinter' + '.txt', 'r')
info_s = []
l_s = 0

data = csv.DictReader(sprinter_file)
for row in data:
    info_s.append(row)
    info_s[l_s]['Cost'] = int(info_s[l_s]['Cost'])
    info_s[l_s]['Points'] = int(info_s[l_s]['Points'])
    l_s += 1
       
sprinter_file.close()

# For TTers
tt_file = open(race_name + '_TT' + '.txt', 'r')
info_tt = []
l_tt = 0

data = csv.DictReader(tt_file)
for row in data:
    info_tt.append(row)
    info_tt[l_tt]['Cost'] = int(info_tt[l_tt]['Cost'])
    info_tt[l_tt]['Points'] = int(info_tt[l_tt]['Points'])
    l_tt += 1
       
tt_file.close()

# For Wildcard
wildcard_file = open(race_name + '_Wildcard' + '.txt', 'r')
info_w = []
l_w = 0

data = csv.DictReader(wildcard_file)
for row in data:
    info_w.append(row)
    info_w[l_w]['Cost'] = int(info_w[l_w]['Cost'])
    info_w[l_w]['Points'] = int(info_w[l_w]['Points'])
    l_w += 1
       
wildcard_file.close()

n_c = len(info_c)
n_s = len(info_s)
n_tt = len(info_tt)
n_all = n_c + n_s + n_tt

In [8]:
set_riders = initialise(n_teams, [])
set_scores = initialise(n_teams, 0)
set_costs = initialise(n_teams, 0)

for c1 in range(n_c-1):
    for c2 in range(c1+1,n_c):
        for s1 in range(n_s-1):
            for s2 in range(s1+1,n_s):
                for tt1 in range(n_tt-1):
                    for tt2 in range(tt1+1,n_tt):
                        for w in range(n_all):
                            if w != c1 and w != c2 and w != s1+n_c and w != s2+n_c and w != tt1+n_c+n_s and w != tt2+n_c+n_s:
                                comb_cost = info_c[c1]['Cost'] + info_c[c2]['Cost'] + info_s[s1]['Cost'] + info_s[s2]['Cost'] + info_tt[tt1]['Cost'] + info_tt[tt2]['Cost'] + info_w[w]['Cost']
                                if comb_cost <= 44:
                                    comb_riders = [info_c[c1]['Name'], info_c[c2]['Name'], info_s[s1]['Name'], info_s[s2]['Name'], info_tt[tt1]['Name'], info_tt[tt2]['Name'], info_w[w]['Name']]
                                    comb_score = info_c[c1]['Points'] + info_c[c2]['Points'] + info_s[s1]['Points'] + info_s[s2]['Points'] + info_tt[tt1]['Points'] + info_tt[tt2]['Points'] + info_w[w]['Points']
                                    unique = checker(comb_riders, set_riders, info_w, n_teams)
                                    for index in range(n_teams):
                                        if comb_score > set_scores[index] and unique:
                                            set_riders = assignment(set_riders, comb_riders, n_teams, index, [])
                                            set_scores = assignment(set_scores, comb_score, n_teams, index, 0)
                                            set_costs = assignment(set_costs, comb_cost, n_teams, index, 0)
                                            unique = False

In [9]:
# This is the new and important bit for this test code.

distribution = np.zeros((n_all,4))

for team in range(100):
    for rider in range(n_riders):
        for row in range(n_all):
            if set_riders[team][rider] == info_w[row]['Name']:
                distribution[row,3] += 1
                break
for team in range(50):
    for rider in range(n_riders):
        for row in range(n_all):
            if set_riders[team][rider] == info_w[row]['Name']:
                distribution[row,2] += 1
                break
for team in range(25):
    for rider in range(n_riders):
        for row in range(n_all):
            if set_riders[team][rider] == info_w[row]['Name']:
                distribution[row,1] += 1
                break
for team in range(10):
    for rider in range(n_riders):
        for row in range(n_all):
            if set_riders[team][rider] == info_w[row]['Name']:
                distribution[row,0] += 1
                break

print(distribution)

[[ 3.  6. 14. 33.]
 [ 0.  2.  5. 15.]
 [ 0.  0.  0.  0.]
 [10. 25. 50. 91.]
 [10. 25. 48. 93.]
 [ 2.  5. 10. 24.]
 [ 1.  6. 11. 21.]
 [ 5. 11. 23. 43.]
 [ 0.  4. 15. 29.]
 [ 8. 18. 31. 63.]
 [ 0.  0.  3.  9.]
 [ 6. 13. 22. 44.]
 [ 4.  8. 15. 32.]
 [ 2.  5. 11. 20.]
 [ 6. 14. 25. 54.]
 [ 3.  8. 17. 34.]
 [ 0.  0.  0.  1.]
 [10. 25. 50. 94.]]


In [11]:
out_file = open(str(race_name) + '_Distribution.txt', 'w')
for row in range(n_all):
    #out_file.write(info_w[row]['Name'] + ' ' + str(int(distribution[row,0])) + '\n')
    #out_file.write(str(int(distribution[row,0])) + '\n')
    out_file.write(info_w[row]['Name'] + ',' + str(int(distribution[row,0])) + ',' + str(int(distribution[row,1])) + ',' + str(int(distribution[row,2])) + ',' + str(int(distribution[row,3])) + '\n')
out_file.close()